Code taken from the official BerTopic github/site: https://maartengr.github.io/BERTopic/getting_started/quickstart/quickstart.html \
Code for reducing outliers based on probability: https://github.com/MaartenGr/BERTopic/issues/529 \
Covert output to features: https://stackoverflow.com/questions/73768683/how-to-get-topic-probs-matrix-in-bertopic-modeling

Visual Novel BerTopic

In [ ]:
pip install bertopic


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 67.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 13.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 119.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 87.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  

In [ ]:
from bertopic import BERTopic
import pandas
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
import numpy as np

In [ ]:
#loading the dataset
df = pandas.read_csv("/content/rogue_lite_final_Bert.csv")

In [ ]:
df.head()

,app_id,app_name,review_text,review_score,App ID,Genre,Owners,Price,Initial Price,Required Age,Tags
0,113200,The Binding of Isaac,"After playing Super meat boy, I started lookin...",0,113200,"Action, Adventure, Indie, RPG","2,000,000 .. 5,000,000",499,499,0,"Action Roguelike: 2341, Rogue-like: 2264, Indi..."
1,113200,The Binding of Isaac,As of writing this I am 500+ hours in and I ha...,0,113200,"Action, Adventure, Indie, RPG","2,000,000 .. 5,000,000",499,499,0,"Action Roguelike: 2341, Rogue-like: 2264, Indi..."
2,113200,The Binding of Isaac,The Binding of Isaac is a small game with very...,0,113200,"Action, Adventure, Indie, RPG","2,000,000 .. 5,000,000",499,499,0,"Action Roguelike: 2341, Rogue-like: 2264, Indi..."
3,113200,The Binding of Isaac,This is pretty much a roguelike dungeon crawle...,0,113200,"Action, Adventure, Indie, RPG","2,000,000 .. 5,000,000",499,499,0,"Action Roguelike: 2341, Rogue-like: 2264, Indi..."
4,113200,The Binding of Isaac,"A stream-lined rouge-like, filled with morbid ...",0,113200,"Action, Adventure, Indie, RPG","2,000,000 .. 5,000,000",499,499,0,"Action Roguelike: 2341, Rogue-like: 2264, Indi..."


In [ ]:
review = df["review_text"].values

In [ ]:
review

In [ ]:
#preperation
embedding_model = SentenceTransformer("all-mpnet-base-v2")
UMAP_model = UMAP(random_state=101)
hdbscan_model = HDBSCAN( metric='euclidean', prediction_data = True, min_samples = 3, min_cluster_size = 20)
vectorizer_model = CountVectorizer(min_df = 5)
ctfidf_model = ClassTfidfTransformer()

In [ ]:
#making topic model
topic_model = BERTopic(embedding_model=embedding_model, umap_model= UMAP_model, hdbscan_model= hdbscan_model, vectorizer_model=vectorizer_model, ctfidf_model=ctfidf_model, calculate_probabilities = True, nr_topics=141)

In [ ]:
topics, probs = topic_model.fit_transform(review)

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,5789,-1_the_and_to_it
1,0,2089,0_it_game_this_you
2,1,1902,1_ship_ships_ftl_crew
3,2,806,2_rebirth_original_isaac_new
4,3,746,3_roguelike_rogue_it_the
...,...,...,...
136,135,22,135_wreck_organs_will_your
137,136,21,136_flame_fire_fires_cats
138,137,20,137_steel_heroes_rpg_turn
139,138,20,138_diehard_dungeon_chest_sword


In [ ]:
topic_model.get_topic(10)

In [ ]:
# Use the "c-TF-IDF" strategy with a threshold
probability_threshold = 0.01
new_topics = [np.argmax(prob) if max(prob) >= probability_threshold else -1 for prob in probs]

In [ ]:
topic_model.update_topics(review, topics=new_topics, vectorizer_model=vectorizer_model)

In [ ]:
topic_model.get_topic_info().sort_values("Count", ascending = False)

,Topic,Count,Name
1,0,6368,0_it_game_the_this
2,1,2490,1_ship_crew_ftl_ships
4,3,1813,3_rogue_roguelike_it_you
6,5,929,5_isaac_binding_items_of
3,2,880,2_rebirth_original_isaac_new
...,...,...,...
140,139,21,139_it_just_hours_for
94,93,20,93_recommend_ma_il_di
93,92,20,92_bob_are_and_to
138,137,20,137_steel_heroes_rpg_turn


In [ ]:
topic_model.get_topic(113)


[('diablo', 0.00839725318282751),
 ('game', 0.006990345981545158),
 ('that', 0.00695554352037096),
 ('the', 0.006797418721737961),
 ('of', 0.006683124495832085),
 ('relics', 0.006611383902808737),
 ('me', 0.006563786847710084),
 ('for', 0.006486313500019013),
 ('to', 0.006480662489575935),
 ('wich', 0.006409932718244899)]

In [ ]:
#getting output features
output_features = pandas.DataFrame(probs)
output_features["sentiment"] = df["review_score"]


In [ ]:
output_features.to_csv("features_RL_Bert_reduction.csv")